# Error Logging

Create ETL Log table

In [ ]:
-- ETL Batch table
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'EtlBatch' AND XTYPE='U')
BEGIN
    CREATE TABLE [dbo].[EtlBatch] (
        [batch_id] [uniqueidentifier] NOT NULL,
		[batch_start_date] [date] NOT NULL,
        [batch_start_time] [time] NOT NULL
    );
END

-- ETL Log table
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'EtlLog' AND XTYPE='U')
BEGIN
    CREATE TABLE [dbo].[EtlLog] (
        [log_id] [int] IDENTITY(1,1) NOT NULL,
        [batch_id] [uniqueidentifier] NULL,
        [error_level] [int] NULL,
		[script_name] [varchar](256) NULL,
        [custom_message] [varchar](1000) NULL,
        [message] [varchar](max) NULL,
        [log_created_date] [date] NOT NULL,
		[log_created_time] [time] NOT NULL
    );
END

-- ETL Log History table
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'EtlLogHistory' AND XTYPE='U')
BEGIN
    CREATE TABLE [dbo].[EtlLogHistory] (
        [log_id] [int] NOT NULL,
        [batch_id] [uniqueidentifier] NULL,
        [error_level] [int] NULL,
		[script_name] [varchar](256) NULL,
        [custom_message] [varchar](1000) NULL,
        [message] [varchar](max) NULL,
        [log_created_date] [date] NOT NULL,
		[log_created_time] [time] NOT NULL
    );
END
GO

Create stored procedure to run at beginning of ETL to manage logging

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[usp_ETL_Preprocess]
AS
BEGIN
	SET NOCOUNT ON;

	-- Create ETL Batch Id
	INSERT INTO [dbo].[EtlBatch] ([batch_id], [batch_start_date], [batch_start_time]) VALUES (NEWID(), GETDATE(), GETDATE());

	-- Move logs to history
	INSERT INTO [dbo].[EtlLogHistory] (
		  [log_id]
		, [batch_id]
		, [error_level]
		, [script_name]
		, [custom_message]
		, [message]
		, [log_created_date]
		, [log_created_time]
	)
	SELECT 
		  [log_id]
		, [batch_id]
		, [error_level]
		, [script_name]
		, [custom_message]
		, [message]
		, [log_created_date]
		, [log_created_time]
	FROM [dbo].[EtlLog]
	ORDER BY [log_id];

	-- Clear logs
	DELETE FROM [dbo].[EtlLog];

END
GO

Create stored procedure to insert error logs into the ETL Log table. This has been tested on SQL Server 2016, 2017, and 2019.

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[LogError]
	@CustomMessage varchar(1000), 
	@ScriptName varchar(1000), 
	@HasBatchId bit = 0	
AS
BEGIN
	SET NOCOUNT ON;

	DECLARE @Batch_ID uniqueidentifier;

	IF @HasBatchId = 1
	BEGIN	
		SET @BATCH_ID = (SELECT TOP 1 batch_id FROM [dbo].[EtlBatch] ORDER BY batch_start_date DESC, batch_start_time DESC);	
	END 

	INSERT INTO [dbo].[EtlLog] (
		  [batch_id]
		, [error_level]
		, [script_name]
		, [custom_message]
		, [message]
		, [log_created_date]
		, [log_created_time]
	)
	SELECT  
		  @Batch_ID AS [batch_id]
		, ERROR_SEVERITY() AS [error_level]
		, @ScriptName AS [script_name]
		, @CustomMessage AS [custom_message]
		, IIF(ERROR_SEVERITY() > 0,
			 CONCAT(
				'ErrorNumber - ', ERROR_NUMBER(),
				' ; ErrorState - ', ERROR_STATE(),
				' ; ErrorProcedure - ', ERROR_PROCEDURE(),
				' ; ErrorLine -' , ERROR_LINE(),
				' ; ErrorMessage - ', ERROR_MESSAGE()
			),
			NULL
		) AS [message]
		, GETDATE() AS [log_created_date] 
		, GETDATE() AS [log_created_time];
END
GO

The code below allows you to use the error log

In [ ]:
-- This belongs at the top of the script
DECLARE @scriptName varchar(200) = (SELECT OBJECT_SCHEMA_NAME(@@PROCID) + '.' + OBJECT_NAME(@@PROCID));

-- Use the following code block to wrap commands you are executing
BEGIN TRY
    -- Replace the following RAISERROR with your code
    RAISERROR ('Error raised in TRY block.', -- Message text.  
        16, -- Severity.  
        1 -- State.  
        );
END TRY  
BEGIN CATCH  
    EXECUTE [dbo].[LogError] @CustomMessage = 'Test Custom Message', @ScriptName = 'Test Script Name', @HasBatchId = 1;
END CATCH;

-- Alternatively you can use this to log things that are not necessarily related to a 
-- batch by passing 0 to @hasBatchId or excluding the argument since it defaults to 0 (false).
BEGIN TRY
    -- Replace the following RAISERROR with your code
    RAISERROR ('Error raised in TRY block.', -- Message text.  
        16, -- Severity.  
        1 -- State.  
        );
END TRY  
BEGIN CATCH  
    EXECUTE [dbo].[LogError] @CustomMessage = 'Test Custom Message', @ScriptName = 'Test Script Name';
END CATCH;
